# Introduction
This notebook will contain code for quickly and procedurally running experiments and comparing their results.

In [ ]:
# package imports
import sys
import os
import subprocess
import numpy as np

# project imports
sys.path.insert(0, os.path.abspath('..'))
from optimization_utils.xml_generate_utils import render_xml
from optimization_utils.result_utils import get_madymo_summary, get_dfs_rmse

In [ ]:
# arguments for running in local environment
# notice that the variable name is capitalized, by convention,
# because we want to indicate that we shouldn't ever change its value
PATH_TO_DATA = fr'../data'
REAL_WORLD_DF = get_madymo_summary(fr"{PATH_TO_DATA}/IIHS_lac.csv")

In [ ]:
# enumerate some experimentional arguments to try
experiment_dicts = [
	{"Friction_Headrest": "1.5", "Friction_Seat": "2"},
	{"Friction_Headrest": "1.5", "Friction_Seat": "1.5"},
	{"Friction_Headrest": "1.5"},
]

In [ ]:
# TODO now I see why you're changing directory (cd) into that path :)
# but I think it's generally not good practice to cd while running code if you can help it,
# because it's very easy to lose where your program is running from, so we should change this a bit
os.chdir(PATH_TO_DATA)

# Run experiment loop
for experiment_dict in experiment_dicts:
    # generate the XML which will be used to run the experiment
    render_xml(
        fr"{PATH_TO_DATA}/Defines.xml",
        fr"{PATH_TO_DATA}/experiment.xml",
        experiment_dict)
    
    # call the madymo subprocess to generate experimental results
    # notice that the experiment will overwrite whatever was output previously - hopefully that's okay?
    subprocess.call(['madymo20201', '-3d', fr"{PATH_TO_DATA}/experiment.xml"])
    
    # read the experimental data spit out by Madymo and compare to the real data
    # TODO I'm assuming I have the filename right, maybe fix it though
    experimental_df = get_madymo_summary("Madymo_lac.csv")
    rmse = get_dfs_rmse(REAL_WORLD_DF, experimental_df)
    
    # record the RMSE of the outputs with the experimental arguments that it came from
    experiment_dict['rmse'] = rmse

In [ ]:
# Look through the experimental results - which parameter set was optimal?
min_dict = min(
    experiment_dicts,
    key = lambda x: float(x['rmse']))

print(min_dict)